In [ ]:
import torchvision.models as models
import torch

import numpy as np
import matplotlib.pyplot as plt


In [ ]:
def get_mobilenet_target_mats():
    target_mats = []
    # Load the model
    model = models.mobilenet_v2(pretrained=True)
    # Put moel into eval mode
    model.eval()
    for layer in model.classifier:
        if isinstance(layer, torch.nn.Linear):
            # Obtain the weights of this layer
            weights = layer.weight.detach().numpy()
            target_mats.append(weights)
    return target_mats

In [ ]:
mats = get_mobilenet_target_mats()

In [ ]:
mats[0]

# Show the matrix

In [ ]:
plt.matshow(mats[0])

# Some different properties

In [ ]:
print('max:',np.abs(mats[0]).max())
print('min:',np.abs(mats[0]).min())

In [ ]:
u, s, vh = np.linalg.svd(mats[0])
plt.plot(s)

In [ ]:
print('max s:',s.max())
print('min s:',s.min())

# Calculate the determinant of submatricies

We want submatrices that do not have full Rank. This means that the determinant in 0.
So lets have a look for different sizes

In [ ]:
def calc_submatdet(A,s):
    dets = np.zeros(np.array(mats[0].shape)-(s-1))
    
    for index, x in np.ndenumerate(dets):
        dets[index]=np.linalg.det(A[index[0]:index[0]+s,index[1]:index[1]+s])
    
    return dets

In [ ]:
dets = calc_submatdet(mats[0],7)
plt.matshow(np.log(abs(dets)))
plt.colorbar()

## Considerations for using a certian value

Possible Values
- smallest $\sigma$
- product of all $\sigma$s -> $\text{det}(MM^T)$


Sum of $\sigma$s is not usefull, as we are interested in the smallest values. 
These are quite irellevant in a prosuct

Upside of a product: not onl the samllest value is considered, but a large $\sigma_1$ can also influence the results

In [ ]:
def calc_submatgramdet(A,s):
    dets = np.zeros(np.array(mats[0].shape)-(s-1))
    
    for index, x in np.ndenumerate(dets):
        M = A[index[0]:index[0]+s,index[1]:index[1]+s]
        dets[index]=np.linalg.det(M@M.T)
    
    return dets

In [ ]:
dets = calc_submatgramdet(mats[0],7)
plt.matshow(np.log10(abs(dets)))
plt.colorbar()

# Calculate the Hankel Rank for all points

In [ ]:
#https://www.mikulskibartosz.name/how-to-display-a-progress-bar-in-jupyter-notebook/
from IPython.display import clear_output

def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

In [ ]:
def calc_ranks(A,s):
    ranks = np.zeros(np.array(mats[0].shape)//s)
    
    cnt = 0
    num = ranks.shape[0]*ranks.shape[1]
    for index, x in np.ndenumerate(ranks):
        
        ranks[index]=np.linalg.matrix_rank(A[index[0]*s:,:index[1]*s+1])
        cnt +=1
        update_progress(cnt/num)
    return ranks

In [ ]:
if False:
    ranks = calc_ranks(mats[0],10)
    with open('ranks.npy', 'wb') as f:
        np.save(f, ranks)
else:    
    with open('ranks.npy', 'rb') as f:
        ranks = np.load(f)
plt.matshow(np.log10(ranks))
plt.colorbar()

In [ ]:
ranks.max()

# Calculate and visualize the spectra of submatrices

General Idea:

Get some Idea how the spectra of the submatrices look like and what a good Hankelrank should be.

Goal:
Get an informed guess how the rank of submatrices behave depending on the shape


In [ ]:
def calc_spectra(A,N=100,shape = np.array([10,10])):  

    Sigmas = np.zeros((N,shape.min()))
    l =np.random.randint(0,size = N,high=A.shape[0]-shape[0])
    j =np.random.randint(0,size = N,high=A.shape[1]-shape[1])
    for i in range(N):
        u, s, vh = np.linalg.svd(A[l[i]:l[i]+shape[0],j[i]:j[i]+shape[1]])
        Sigmas[i,:]=s
    return Sigmas
    

In [ ]:
N=200
Sigmas = calc_spectra(mats[0],N=N,shape=np.array([5,5]))
plt.figure()
plt.scatter(Sigmas.flatten(),(np.linspace(0,1,N)*np.ones((Sigmas.shape[1],1))).T.flatten())
#scatter plot: each matrix gets seperate own line
plt.figure()
v =plt.hist(Sigmas.flatten(), bins=100)

In [ ]:
N = 30
fig = plt.figure(figsize=(16,16))
axs = fig.subplots(6, 6)

shapes = np.array([5,10,20,50,100,200])
for i,a in np.ndenumerate(axs):
    shape=np.array([shapes[i[1]],shapes[i[0]]])
    #add 2 plots to get an idea of the variance
    Sigmas = calc_spectra(mats[0],N=N,shape=shape)
    a.hist(Sigmas.flatten(), bins=100,alpha = 0.5)
    Sigmas = calc_spectra(mats[0],N=N,shape=shape)
    a.hist(Sigmas.flatten(), bins=100,alpha = 0.5)
    text="h="+str(shape[0])+" w="+str(shape[1])
    a.set_title(text, fontsize=10,x=0.6, y=0.8)

In [ ]:
N = 10
fig = plt.figure(figsize=(16,16))
axs = fig.subplots(6, 6)

def calc_spectra_hankel(A,shape = np.array([10,10])):  
    #Sigmas = np.zeros((N,shape.min()))
    #for i in range(N):
    u, s, vh = np.linalg.svd(A[-shape[0]:,:shape[1]])
        #Sigmas[i,:]=s
    return s

shapes = np.array([50,100,200,250,500,800])
for i,a in np.ndenumerate(axs):
    shape=np.array([shapes[i[1]],shapes[i[0]]])
    #add 2 plots to get an idea of the variance
    #Sigmas = calc_spectra_hankel(mats[0],N=N,shape=shape,direc=+1)
    #a.hist(Sigmas.flatten(), bins=100,alpha = 0.5)
    Sigmas = calc_spectra_hankel(mats[0],shape=shape)
    a.hist(Sigmas.flatten(), bins=100,alpha = 0.5)
    text="h="+str(shape[0])+" w="+str(shape[1])
    a.set_title(text, fontsize=10,x=0.6, y=0.8)